In [3]:
import pandas as pd
df_loaded = pd.read_csv('../dataset/clean/clean_sak_occupation_full.csv', 
                        sep=',', 
                        quotechar='"', 
                        encoding='utf-8',dtype=str)
df_loaded.head()

,text_description,kbli_code,kbji_code
0,aktivitas: kepala desa. produk: jasa pemerinta...,84119,1113
1,aktivitas: mencangkul dan menanam padi. produk...,01122,6111
2,aktivitas: melayani administrasi di kantor kec...,84119,1213
3,aktivitas: administrasi smp 1 negeri teupah se...,85491,4131
4,aktivitas: jualan bahan makanan. produk: semba...,47112,5211


In [5]:
df_kbli=df_loaded[["text_description","kbli_code"]]

df_kbli.head()

,text_description,kbli_code
0,aktivitas: kepala desa. produk: jasa pemerinta...,84119
1,aktivitas: mencangkul dan menanam padi. produk...,01122
2,aktivitas: melayani administrasi di kantor kec...,84119
3,aktivitas: administrasi smp 1 negeri teupah se...,85491
4,aktivitas: jualan bahan makanan. produk: semba...,47112


In [6]:
df_kbli["kbli_code"].nunique(), len(df_kbli)


(1756, 2085667)